A/B Hypothesis Testing
Claim Frequency  and Claim Severity   

Margin = TotalPremium - TotalClaims

In [1]:
import pandas as pd
from scipy import stats

In [2]:
df=pd.read_csv('MachineLearningRating_v3.txt',sep='|')

C:\Users\Hp\AppData\Local\Temp\ipykernel_17452\3239025493.py:1: DtypeWarning: Columns (0,1,3,14,18,20,23,24,25,27,29,32,37,38,39,41,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('MachineLearningRating_v3.txt',sep='|')


In [30]:
df.head()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,HasClaim
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0,0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.0,0.0,0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.84807,0.0,0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.0,0.0,0


H₀:There are no risk differences across provinces 

In [80]:
df.dropna()

,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,HasClaim,want,ClaimSeverity,margin


In [84]:

import numpy as np
df['TotalClaims']=pd.to_numeric(df['TotalClaims'],errors='coerce')
df['ClaimSeverity']=pd.to_numeric(df['ClaimSeverity'])
df['HasClaim']=df['TotalClaims'].apply(lambda x:1 if x >0 else 0)
df['ClaimSeverity']=df.apply(lambda x: x['TotalClaims'] if x['HasClaim'] >0 else np.nan ,axis=1)
df['TotalPremium']=pd.to_numeric(df['TotalPremium'],errors='coerce')
df['margin']=df['TotalPremium']-df['TotalClaims']



H₀:There are no risk differences across provinces 

In [120]:
df['Province'].value_counts().head(2)

Province
Gauteng         787730
Western Cape    341592
Name: count, dtype: int64

In [136]:
from scipy import stats
group_A=df[df['Province']=='Gauteng']
group_b=df[df['Province']=='Western Cape']
# has cliam is binary(0,1) so i use chisquaretest the rest ttest for 2 groups
group_A=group_A.dropna(subset=['ClaimSeverity','margin'])
group_b=group_b.dropna(subset=['ClaimSeverity','margin'])

contigency_table=pd.crosstab(df['Province'].isin(['Gauteng', 'Western Cape']),df['HasClaim'])
_,p_val_cliam,_,_=stats.chi2_contingency(contigency_table)
t_stat,p_val_severity=stats.ttest_ind(group_A['ClaimSeverity'],group_b['ClaimSeverity'])
print(f"p val for claim {p_val_cliam}")
print(f'p val for severity  {p_val_severity}')
print(f"tstat",{t_stat})
if p_val_severity <0.05 and p_val_cliam <0.05:
    print("we gona regect the null hypothesis  Western Cape has higher frequency  and severity of Cliams con: Total Premimum must be increase")
else:
    print("ho is correct ")


p val for claim 1.9751829406106745e-10
p val for severity  0.0003136131508135969
tstat {np.float64(-3.607517383360546)}
we gona regect the null hypothesis  Western Cape has higher frequency  and severity of Cliams con: Total Premimum must be increase


In [118]:
top_two_modes = df['PostalCode'].value_counts().head(2)
print(top_two_modes)

PostalCode
2000    266462
122      97471
Name: count, dtype: int64


H₀:There are no risk differences between zip codes 

In [137]:
from scipy import stats
import pandas as pd
group_A = df[df['PostalCode'] == 122].copy()
group_B = df[df['PostalCode'] == 2000].copy()
group_A['ClaimSeverity'] = pd.to_numeric(group_A['ClaimSeverity'], errors='coerce')
group_B['ClaimSeverity'] = pd.to_numeric(group_B['ClaimSeverity'], errors='coerce')
group_A = group_A.dropna(subset=['HasClaim', 'ClaimSeverity'])
group_B = group_B.dropna(subset=['HasClaim', 'ClaimSeverity'])
chi_table = pd.crosstab(df['PostalCode'].isin(['122','2000']),df['HasClaim'])
_, p_val_claim, _, _ = stats.chi2_contingency(chi_table)
t_stat_severity, p_val_severity = stats.ttest_ind(group_A['ClaimSeverity'], group_B['ClaimSeverity'])

print(f"p-value for HasClaim (Chi-square): {p_val_claim}")
print(f"p-value for ClaimSeverity (t-test): {p_val_severity}")
if p_val_severity and p_val_severity<0.05 :
    print("we gona regect the null hypothesis  postalcode 2000 has higher frequency  and severity of Cliams con: Total Premimum must be increase")
else:
    print("small number of sample we can not reject null hyposthesis ")


p-value for HasClaim (Chi-square): 0.41705654044475926
p-value for ClaimSeverity (t-test): 0.5534150456258154
small number of sample we can not reject null hyposthesis 


There are no significant margin (profit) difference between zip codes 

In [138]:
tstat,p_val_cliam=stats.ttest_ind(group_A['margin'],group_b['margin'])
print(f"p val for claim {p_val_claim}")
 
if p_val_cliam <0.05 :
    print("we gona regect the null hypothesis  Postalcode 2000  has higher frequency claim   and severity of Cliams con: Total Premimum must be increase")
else:
    print("small number of sample we can not reject null hyposthesis ")

p val for claim 0.41705654044475926
we gona regect the null hypothesis  Postalcode 2000  has higher frequency claim   and severity of Cliams con: Total Premimum must be increase


H₀:There are not significant risk difference between Women and Men

In [ ]:
group_A=df[df['Gender']=='Male'].copy()
group_b=df[df['Gender']=='Female'].copy()
group_A['ClaimSeverity'] = pd.to_numeric(group_A['ClaimSeverity'], errors='coerce')
group_b['ClaimSeverity'] = pd.to_numeric(group_b['ClaimSeverity'], errors='coerce')
group_A = group_A.dropna(subset=['HasClaim', 'ClaimSeverity'])
group_b = group_b.dropna(subset=['HasClaim', 'ClaimSeverity'])
chi2_cont_table=pd.crosstab(df['Gender'].isin(['Male','Female']),df['HasClaim'])
_,p_val_cliam,_,_=stats.chi2_contingency(chi2_cont_table)
t_stat_severity,p_val_severity=stats.ttest_ind(group_A['ClaimSeverity'],group_b['ClaimSeverity'])
print(f"p val for claim {p_val_cliam}")
print(f'p val for severity  {p_val_severity}')
print(f"{t_stat}")
if p_val_cliam < 0.05:
    print("Reject null: Claim frequency differs by gender  Gender make significant diffrence ")
else:
    print("Fail to reject: Claim frequency similar by gender")

if p_val_severity < 0.05:
    print("Reject null: Claim severity differs by gender")
else:
    print("Fail to reject: Claim severity similar by gender")


p val for claim 0.00021550806538669622
p val for severity  0.5521797732352693
-3.607517383360546
small number of sample we can not reject null hyposthesis 
